<a href="https://colab.research.google.com/github/zentho/zentho/blob/main/Copy_of_CS189_HW_TSNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 189 Homework 4 T-SNE
**Note:** before starting this notebook, please save a copy of it to your own google drive, or your changes will not persist.

In this problem, you will explore one way in which an ML engineer might try to interpret what the neural network they have just trained is doing. It turns out that t-SNE can come in handy here not just as a data visualization tool, but also as a *feature* visualization tool. Neural nets are, after all, trying to learn good features of the data for prediction.

You will use scikit-learn's TSNE functionality for this problem, so it would be a good idea to look at that documentation. Your deliverables will be your code in this notebook as well as all plots that you produce here.



In [ ]:
# Imports for pytorch
import numpy as np
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from sklearn.manifold import TSNE
import matplotlib
from matplotlib import pyplot as plt
import tqdm.notebook as tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)

transform = torchvision.transforms.Compose(
          [torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

training_data = torchvision.datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

test_data = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform)

batch_size = 4
trainloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Feel free to visualize the data to get a sense of what the dataset looks like (note that the images have been normalized):

In [ ]:
images = [training_data[i][0] for i in range(9)]
plt.imshow(torchvision.utils.make_grid(torch.stack(images), nrow=3, padding=5).numpy().transpose((1, 2, 0)))

Part (a): Take the first 1000 images in the training dataset and perform t-SNE on the flattened images. Plot the t-SNE embeddings and color-code them by the class of each data point.

In [ ]:
### Part (a) ###
### YOUR CODE HERE ###


Part (b): Find the test accuracy of the neural network provided.



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 128, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(128, 128, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool1 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(128, 256, 3, 1, 1)
        self.bn3 = nn.BatchNorm2d(256)
        self.linear1 = nn.Linear(256 * 16 * 16, 256)
        self.bn_l1 = nn.BatchNorm1d(256)
        self.linear2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.pool1(x)
        x = self.bn3(F.relu(self.conv3(x)))

        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x


# Loading model
net = Net().to(device)
model_save_name = 'cifar10_classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" # Change path as necessary!
net = torch.load(path)
net.eval()

In [ ]:
### Part (b) ###
### YOUR CODE HERE ###


Part (c): For the following parts, we will make use of *hook* functions to save the outputs of particular layers of the model during a forward pass. We have provided a function below describing its usage. Use the function to obtain the set of outputs from net.conv3 for the first 1000 images of the training dataset as inputs. Then, run t-SNE on those outputs. Plot the t-SNE embeddings and color-code them by the class of each data point.

For reference, the neural network layers are: \\
net.conv1 \\
net.conv2 \\
net.conv3 \\
net.linear1 \\
net.linear2 \\

In [ ]:
class SaveFeatures():
     features=None
     def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
     def hook_fn(self, module, input, output): self.features = ((output.cpu()).data).numpy()
     def remove(self): self.hook.remove()

In [ ]:
# Example: get_features_from_layer(net.conv1)
def get_features_from_layer(layer):
  activated_features = SaveFeatures(layer)
  return activated_features

In [ ]:
### Part (c) ###
### YOUR CODE HERE ###
## Hint: Call get_features_from_layer() and use the 'features' attribute of the SaveFeatures class


Part (d): Do the same as part (c) except for the first and second linear layers of the network.

In [ ]:
### Part (d) ###
### YOUR CODE HERE ###


Congrats! You made it to the end.